**This notebook covered by the following [license](License.ipynb)  This note must not be removed**

# Ingress

- [Must be enabled](IngressStart.ipynb)
- [Service Networking Concept](https://kubernetes.io/docs/concepts/services-networking/ingress/)
- Implemented by an [Ingress Controller](https://kubernetes.io/docs/concepts/services-networking/ingress-controllers/)
- Implementation dependent [Overview by Yitaek Hwang](https://medium.com/swlh/kubernetes-ingress-controller-overview-81abbaca19ec)
- Can include [External Oauth 2 ](https://kubernetes.github.io/ingress-nginx/examples/auth/oauth-external-auth/)
- Most popular [NGinx Ingress Controller](https://docs.nginx.com/nginx-ingress-controller/)

In [8]:
kubectl create deployment web --image=gcr.io/google-samples/hello-app:1.0

deployment.apps/web created


In [10]:
kubectl expose deployment web --type=NodePort --port=8080

service/web exposed


In [14]:
kubectl get service web 

NAME   TYPE       CLUSTER-IP      EXTERNAL-IP   PORT(S)          AGE
web    NodePort   10.106.226.19   <none>        8080:31338/TCP   2m19s


In [18]:
NODEPORT=$(kubectl get service web  -o jsonpath='{.spec.ports[0].nodePort}')
echo $NODEPORT

31338


In [20]:
curl $(hostname -i):$NODEPORT

Hello, world!
Version: 1.0.0
Hostname: web-79d88c97d6-chhxn


In [32]:
kubectl apply -f - <<EOF
apiVersion: networking.k8s.io/v1
kind: Ingress
metadata:
  name: example-ingress
  annotations:
    nginx.ingress.kubernetes.io/rewrite-target: /$1
spec:
  rules:
    - host: $(hostname -i).nip.io
      http:
        paths:
          - path: /
            pathType: Prefix
            backend:
              service:
                name: web
                port:
                  number: 8080
EOF

ingress.networking.k8s.io/example-ingress unchanged


In [33]:
kubectl get ingress example-ingress -o yaml

apiVersion: networking.k8s.io/v1
kind: Ingress
metadata:
  annotations:
    kubectl.kubernetes.io/last-applied-configuration: |
      {"apiVersion":"networking.k8s.io/v1","kind":"Ingress","metadata":{"annotations":{"nginx.ingress.kubernetes.io/rewrite-target":"/"},"name":"example-ingress","namespace":"default"},"spec":{"rules":[{"host":"192.168.49.2.nip.io","http":{"paths":[{"backend":{"service":{"name":"web","port":{"number":8080}}},"path":"/","pathType":"Prefix"}]}}]}}
    nginx.ingress.kubernetes.io/rewrite-target: /
  creationTimestamp: "2021-01-24T16:39:50Z"
  generation: 2
  managedFields:
  - apiVersion: networking.k8s.io/v1
    fieldsType: FieldsV1
    fieldsV1:
      f:metadata:
        f:annotations:
          .: {}
          f:kubectl.kubernetes.io/last-applied-configuration: {}
          f:nginx.ingress.kubernetes.io/rewrite-target: {}
      f:spec:
        f:rules: {}
    manager: kubectl-client-side-apply
    operation: Update
    time: "2021-01-24T16:39:50Z"
  - apiVersi

In [23]:
kubectl describe ingress my-ingress

Name:             my-ingress
Namespace:        default
Address:          192.168.49.2
Default backend:  default-http-backend:80 (<error: endpoints "default-http-backend" not found>)
Rules:
  Host        Path  Backends
  ----        ----  --------
  *           
              /helloweb   helloweb:80 (<error: endpoints "helloweb" not found>)
Annotations:  <none>
Events:
  Type    Reason  Age   From                      Message
  ----    ------  ----  ----                      -------
  Normal  CREATE  19m   nginx-ingress-controller  Ingress default/my-ingress
  Normal  UPDATE  19m   nginx-ingress-controller  Ingress default/my-ingress


In [35]:
INGRESS_HOST=$(kubectl get ingress example-ingress -o jsonpath='{.spec.rules[0].host}')
echo $INGRESS_HOST

192.168.49.2.nip.io


In [36]:
curl $INGRESS_HOST/helloweb

Hello, world!
Version: 1.0.0
Hostname: web-79d88c97d6-chhxn


## Create **more** services
We can say hello world in different ways

In [37]:
kubectl create deployment web2 --image=gcr.io/google-samples/hello-app:2.0


deployment.apps/web2 created


In [38]:
kubectl expose deployment web2 --port=8080 --type=NodePort


service/web2 exposed


In [41]:
kubectl apply -f - <<EOF
apiVersion: networking.k8s.io/v1
kind: Ingress
metadata:
  name: example-ingress
  annotations:
    nginx.ingress.kubernetes.io/rewrite-target: /$1
spec:
  rules:
    - host: $(hostname -i).nip.io
      http:
        paths:
          - path: /
            pathType: Prefix
            backend:
              service:
                name: web
                port:
                  number: 8080
          - path: /v2
            pathType: Prefix
            backend:
              service:
                name: web2
                port:
                 number: 8080
EOF

ingress.networking.k8s.io/example-ingress configured


In [42]:
curl $INGRESS_HOST/v2

Hello, world!
Version: 2.0.0
Hostname: web2-5d47994f45-htxjm


In [39]:
kubectl get all

NAME                        READY   STATUS    RESTARTS   AGE
pod/web-79d88c97d6-chhxn    1/1     Running   0          19m
pod/web2-5d47994f45-htxjm   1/1     Running   0          13s

NAME                 TYPE        CLUSTER-IP       EXTERNAL-IP   PORT(S)          AGE
service/kubernetes   ClusterIP   10.96.0.1        <none>        443/TCP          26h
service/web          NodePort    10.106.226.19    <none>        8080:31338/TCP   18m
service/web2         NodePort    10.107.254.143   <none>        8080:30654/TCP   1s

NAME                   READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/web    1/1     1            1           19m
deployment.apps/web2   1/1     1            1           13s

NAME                              DESIRED   CURRENT   READY   AGE
replicaset.apps/web-79d88c97d6    1         1         1       19m
replicaset.apps/web2-5d47994f45   1         1         1       13s


## TLS

In [45]:
mkdir -p openssl
cd openssl


In [46]:
DOMAIN=$INGRESS_HOST
CONF=${DOMAIN}.conf
SECRET=${DOMAIN}.yaml
KEY=${DOMAIN}.key
CRT=${DOMAIN}.crt

echo $DOMAIN

192.168.49.2.nip.io


In [47]:
pwd

/minikube-host/training/notebooks/openssl


In [51]:
ls

192.168.49.2.nip.io.conf


In [52]:
test  ! -f ${CONF}  && cat > ${CONF}  <<EOF
[ req ]
default_bits       = 2048
default_md         = sha512
default_keyfile    = ${KEYFILE}
prompt             = no
encrypt_key        = no
distinguished_name = req_distinguished_name
# distinguished_name
[ req_distinguished_name ]
countryName            = DE
localityName           = Berlin
organizationName       = nip.io
organizationalUnitName = K8S Demo Department
commonName             = *.${DOMAIN}
emailAddress           = nowhere@${DOMAIN}


EOF

In [53]:
echo $KEY $CONF $CRT

test  ! -f ${KEY} && openssl req -config ${CONF} -newkey rsa:2048 -nodes -keyout ${KEY} -x509 -out ${CRT}

192.168.49.2.nip.io.key 192.168.49.2.nip.io.conf 192.168.49.2.nip.io.crt
Generating a RSA private key
....................................................................................................+++++
................+++++
writing new private key to '192.168.49.2.nip.io.key'
-----


In [54]:
test ! -f ${SECRET} && cat > ${SECRET}<<EOF
apiVersion: v1
kind: Secret
metadata:
 name: wildcard.tls
 namespace: default
type: kubernetes.io/tls
data:
 tls.crt: `cat ${CRT} | base64 -w0`
 tls.key: `cat ${KEY} | base64 -w0`
EOF


In [55]:
cat $SECRET

apiVersion: v1
kind: Secret
metadata:
 name: wildcard.tls
 namespace: default
type: kubernetes.io/tls
data:
 tls.crt: LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUR1ekNDQXFNQ0ZEMENheEgyb2Z2aWt2Z2h2V2c2bnFTaDgrdHlNQTBHQ1NxR1NJYjNEUUVCRFFVQU1JR1oKTVFzd0NRWURWUVFHRXdKRVJURVBNQTBHQTFVRUJ3d0dRbVZ5YkdsdU1ROHdEUVlEVlFRS0RBWnVhWEF1YVc4eApIREFhQmdOVkJBc01FMHM0VXlCRVpXMXZJRVJsY0dGeWRHMWxiblF4SGpBY0JnTlZCQU1NRlNvdU1Ua3lMakUyCk9DNDBPUzR5TG01cGNDNXBiekVxTUNnR0NTcUdTSWIzRFFFSkFSWWJibTkzYUdWeVpVQXhPVEl1TVRZNExqUTUKTGpJdWJtbHdMbWx2TUI0WERUSXhNREV5TkRFMk5UWXpOMW9YRFRJeE1ESXlNekUyTlRZek4xb3dnWmt4Q3pBSgpCZ05WQkFZVEFrUkZNUTh3RFFZRFZRUUhEQVpDWlhKc2FXNHhEekFOQmdOVkJBb01CbTVwY0M1cGJ6RWNNQm9HCkExVUVDd3dUU3poVElFUmxiVzhnUkdWd1lYSjBiV1Z1ZERFZU1Cd0dBMVVFQXd3VktpNHhPVEl1TVRZNExqUTUKTGpJdWJtbHdMbWx2TVNvd0tBWUpLb1pJaHZjTkFRa0JGaHR1YjNkb1pYSmxRREU1TWk0eE5qZ3VORGt1TWk1dQphWEF1YVc4d2dnRWlNQTBHQ1NxR1NJYjNEUUVCQVFVQUE0SUJEd0F3Z2dFS0FvSUJBUURzd3hoTUsvRzEySll4Cm5aQ0dhYW5aS0VmYXZwR2dIUkR1SXB3MXVnTUhYZmd4UUpUZjh0a2s0a1RxeFZEMU

In [56]:
kubectl apply -f $SECRET

secret/wildcard.tls created


In [57]:
cd ..

In [58]:
pwd

/minikube-host/training/notebooks


In [61]:
kubectl delete ingress example-ingress

ingress.networking.k8s.io "example-ingress" deleted


In [62]:
kubectl get ingress

No resources found in default namespace.


In [79]:
kubectl apply -f - <<EOF
apiVersion: networking.k8s.io/v1
kind: Ingress
metadata:
  name: website
spec:
  tls:
    - hosts:
      - demo.$DOMAIN
      - info.$DOMAIN
      secretName: wildcard.tls
  rules:
  - host: info.$DOMAIN
    http:
      paths:
      - path: /web
        pathType: Prefix
        backend:
          service:
            name: web
            port:
              number: 8080
      - path: /web/v2
        pathType: Prefix
        backend:
          service:
            name: web2
            port:
              number: 8080
  - host: demo.$DOMAIN
    http:
      paths:
      - path: /
        pathType: Prefix
        backend:
          service:
            name: web2
            port:
              number: 8080
EOF

ingress.networking.k8s.io/website created


In [84]:
curl -k https://info.$DOMAIN/web
curl -k https://info.$DOMAIN/web/v2
curl -k https://demo.$DOMAIN/


Hello, world!
Version: 1.0.0
Hostname: web-79d88c97d6-chhxn
Hello, world!
Version: 2.0.0
Hostname: web2-5d47994f45-htxjm
Hello, world!
Version: 2.0.0
Hostname: web2-5d47994f45-htxjm


In [85]:
kubectl get ingress

NAME      CLASS    HOSTS                                               ADDRESS        PORTS     AGE
website   <none>   info.192.168.49.2.nip.io,demo.192.168.49.2.nip.io   192.168.49.2   80, 443   2m21s


In [86]:
kubectl delete ingress website

ingress.networking.k8s.io "website" deleted
